In [64]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler



In [49]:
train =pd.read_csv(r"E:\Download\house price\train (2).csv")
test =pd.read_csv(r"E:\Download\house price\test (2).csv")

In [50]:
train.head()
test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [52]:

missing_values = train.isnull().sum().sort_values(ascending=False)
print(missing_values[missing_values > 0])  


PoolQC          1453
MiscFeature     1406
Alley           1369
Fence           1179
MasVnrType       872
FireplaceQu      690
LotFrontage      259
GarageYrBlt       81
GarageCond        81
GarageType        81
GarageFinish      81
GarageQual        81
BsmtFinType2      38
BsmtExposure      38
BsmtQual          37
BsmtCond          37
BsmtFinType1      37
MasVnrArea         8
Electrical         1
dtype: int64


In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [9]:
missing_values = train.isnull().sum()
missing_values = missing_values[missing_values > 0].sort_values(ascending=False)
print(missing_values)

PoolQC          1453
MiscFeature     1406
Alley           1369
Fence           1179
MasVnrType       872
FireplaceQu      690
LotFrontage      259
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
BsmtFinType2      38
BsmtExposure      38
BsmtFinType1      37
BsmtCond          37
BsmtQual          37
MasVnrArea         8
Electrical         1
dtype: int64


In [11]:
train.shape

(1460, 81)

In [54]:
num_imputer = SimpleImputer(strategy="median")
train_numeric = train.select_dtypes(include=[np.number])
train_numeric_imputed = pd.DataFrame(num_imputer.fit_transform(train_numeric), columns=train_numeric.columns)

In [55]:
cat_imputer = SimpleImputer(strategy="most_frequent")
train_categorical = train.select_dtypes(exclude=[np.number])
train_categorical_imputed = pd.DataFrame(cat_imputer.fit_transform(train_categorical), columns=train_categorical.columns)

In [56]:
train_processed = pd.concat([train_numeric_imputed, train_categorical_imputed], axis=1)

In [58]:

train_numeric = train.select_dtypes(include=[np.number]).drop(["SalePrice"], axis=1)

num_imputer = SimpleImputer(strategy="median")
train_numeric_imputed = pd.DataFrame(num_imputer.fit_transform(train_numeric), columns=train_numeric.columns)


test_numeric = test.select_dtypes(include=[np.number])
test_numeric_imputed = pd.DataFrame(num_imputer.transform(test_numeric), columns=test_numeric.columns)


In [61]:

train_categorical = train.select_dtypes(include=["object"])
test_categorical = test.select_dtypes(include=["object"])


cat_imputer = SimpleImputer(strategy="most_frequent")
train_categorical_imputed = pd.DataFrame(cat_imputer.fit_transform(train_categorical), columns=train_categorical.columns)
test_categorical_imputed = pd.DataFrame(cat_imputer.transform(test_categorical), columns=test_categorical.columns)

train_encoded = pd.get_dummies(train_categorical_imputed)
test_encoded = pd.get_dummies(test_categorical_imputed)


train_encoded, test_encoded = train_encoded.align(test_encoded, join="left", fill_value=0)


In [62]:
X_train = pd.concat([train_numeric_imputed, train_encoded], axis=1)
X_test = pd.concat([test_numeric_imputed, test_encoded], axis=1)

y_train = train["SalePrice"]


In [65]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [66]:
from xgboost import XGBRegressor


xgb_model = XGBRegressor(n_estimators=500, learning_rate=0.05, max_depth=6, random_state=42)


xgb_model.fit(X_train_scaled, y_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=500, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [67]:
y_pred = xgb_model.predict(X_test_scaled)


In [73]:
print("Length of y_pred:", len(y_pred))
print("Length of test['Id']:", len(test["Id"]))


Length of y_pred: 1460
Length of test['Id']: 1459


In [74]:
y_pred = y_pred[:len(test)]


In [77]:
submission = pd.DataFrame({
    "Id": test["Id"].values[:len(y_pred)],  
    "SalePrice": y_pred
})

submission.to_csv("submission.csv", index=False)

submission


,Id,SalePrice
0,1461,125436.242188
1,1462,155699.718750
2,1463,189161.703125
3,1464,188474.312500
4,1465,197188.031250
...,...,...
1454,2915,81178.507812
1455,2916,79646.179688
1456,2917,148859.953125
1457,2918,114245.210938
